In [1]:
# | default_exp classes.DomoDatacenter

In [2]:
# | exporti
import asyncio
from dataclasses import dataclass
from typing import Union, List
from enum import Enum

import httpx

from nbdev.showdoc import patch_to

import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoDataset as dmds
import domolibrary.classes.DomoAccount as dma
import domolibrary.classes.DomoCard as dmc

import domolibrary.routes.datacenter as datacenter_routes
import domolibrary.routes.card as card_routes

In [3]:
#| hide
import os
import pandas as pd

from domolibrary.routes.datacenter import (
    generate_search_datacenter_filter,
    Datacenter_Filter_Field_Enum,
    Datacenter_Filter_Field_Certification_Enum,
)


In [4]:
# | export


@dataclass
class DomoDatacenter:
    "class for quering entities in the datacenter"
    auth: dmda.DomoAuth = None

In [5]:
# | exporti


@patch_to(DomoDatacenter, cls_method=True)
async def search_datacenter(
    cls,
    auth: dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    body: dict = None,  # either pass a body or generate a body in the function using search_text, entity_type, and additional_filters parameters
    search_text=None,
    # can accept one value or a list of values
    entity_type: Union[str, list] = "dataset",
    additional_filters_ls=None,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> list:

    res = await datacenter_routes.search_datacenter(
        auth=auth,
        maximum=maximum,
        body=body,
        session=session,
        search_text=search_text,
        entity_type=entity_type,
        additional_filters_ls=additional_filters_ls,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return res.response

#### sample implementation of search_datacenter


In [6]:
#| hide
# import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    datacenter_routes.generate_search_datacenter_filter(
        "dataprovidername_facet", "Jupyter Data"
    )
]

res = await DomoDatacenter.search_datacenter(
    auth=token_auth,
    additional_filters_ls=additional_filters_ls,
    search_text="*kb*",
    entity_type=datacenter_routes.Datacenter_Enum.DATASET.value,
)

pd.DataFrame(res[0:5])

warning this token has not been validated by who_am_i, run get_auth_token first


,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,pdpEnabled,used,owners,ownersLocalized,cloudName,cloudId,systemTagFields,winnerText,ownedByType,customer
0,dataset,04c1574e-c8be-4721-9846-c6ffa491144b,"{'indexName': None, 'databaseId': '04c1574e-c8...",1668379680000,1711219384000,1711219785357,{},English,False,11.0,...,True,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson1', 'count': 1}",Domo,domo,[system_tags],domo_kbs,USER,mmmm-0012-0200


In [7]:
#| hide
# import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await DomoDatacenter.search_datacenter(
    auth=token_auth,
    entity_type=datacenter_routes.Datacenter_Enum.CARD.value,
)

pd.DataFrame(res[0:5])

warning this token has not been validated by who_am_i, run get_auth_token first


,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,dataSourceIds,owners,ownersLocalized,keyResults,dataSubscriptions,systemTagFields,winnerText,ownedByType,customer,description
0,card,1036070144,"{'indexName': None, 'databaseId': '1036070144'...",1705541763000,1705541764000,1705542136876,{},English,False,3009.00000,...,[fff33fa8-bc96-4754-92ad-5a2f8789717f],"[{'id': '1734948150', 'type': 'USER', 'display...","{'localizedMessage': 'Thomas Bruno', 'count': 1}",[],"[{'name': 'big_number', 'dataSourceId': 'fff33...",[system_tags],Salesforce,USER,mmmm-0012-0200,NaN
1,card,1693394557,"{'indexName': None, 'databaseId': '1693394557'...",1624546022000,1624548286000,1690401762634,{},English,False,3009.00000,...,[4fc29727-43a4-47ad-94a0-d0021dad8f40],"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson1', 'count': 1}",[],"[{'name': 'big_number', 'dataSourceId': '4fc29...",[system_tags],gov_datasetEfficiency,USER,mmmm-0012-0200,
2,card,1396128897,"{'indexName': None, 'databaseId': '1396128897'...",1710807711000,1710808461000,1710809296465,{},English,False,89.40858,...,[341fdcbd-de8d-4a5e-bf19-f465a43131c8],"[{'id': '68216396', 'type': 'USER', 'displayNa...","{'localizedMessage': 'Elliott Leonard', 'count...",[],"[{'name': 'main', 'dataSourceId': '341fdcbd-de...",[system_tags],Form and Dataset Brick,USER,mmmm-0012-0200,NaN
3,card,1569597825,"{'indexName': None, 'databaseId': '1569597825'...",1710806580000,1710807647000,1710809296464,{},English,False,83.48876,...,"[db8b41a5-ee6d-4bfc-86d6-b6c11786ce9f, fff33fa...","[{'id': '68216396', 'type': 'USER', 'displayNa...","{'localizedMessage': 'Elliott Leonard', 'count...",[],"[{'name': 'main', 'dataSourceId': 'db8b41a5-ee...",[system_tags],Searchable Filter App (Multiple Columns),USER,mmmm-0012-0200,NaN
4,card,1601825647,"{'indexName': None, 'databaseId': '1601825647'...",1616965358000,1623843276000,1711216181500,{},English,False,73.56778,...,[ca6f4b8a-ad46-4ea7-be9f-39aca37b5b25],"[{'id': '55874022', 'type': 'USER', 'displayNa...","{'localizedMessage': 'Grant Smith', 'count': 1}",[],"[{'name': 'main', 'dataSourceId': 'ca6f4b8a-ad...",[system_tags],Start / End of Week or Month,USER,mmmm-0012-0200,https://dojo.domo.com/discussion/comment/52736...


In [8]:
# | exporti
@patch_to(DomoDatacenter, cls_method=True)
async def search_datasets(
    cls,
    auth=dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    search_text=None,
    # can accept one value or a list of values
    additional_filters_ls=None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dmds.DomoDataset]:

    json_list = await cls.search_datacenter(
        auth=auth,
        maximum=maximum,
        search_text=search_text,
        entity_type=datacenter_routes.Datacenter_Enum.DATASET.value,
        additional_filters_ls=additional_filters_ls,
        return_raw=return_raw,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or len(json_list) == 0:
        return json_list

    return await ce.gather_with_concurrency(
        n=20,
        *[
            dmds.DomoDataset.get_from_id(
                dataset_id=json_obj.get("databaseId"),
                auth=auth,
                debug_api=debug_api,
                session=session,
            )
            for json_obj in json_list
        ]
    )

#### sample implementation search_datasets


In [9]:
#| eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    datacenter_routes.generate_search_datacenter_filter(
        "dataprovidername_facet", "Jupyter Data"
    )
]

await DomoDatacenter.search_datasets(
    auth=token_auth,
    search_text="*kb*",
    additional_filters_ls=additional_filters_ls,
)

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoDataset(id='04c1574e-c8be-4721-9846-c6ffa491144b', display_type='domo-jupyterdata', data_provider_type='domo-jupyterdata', name='domo_kbs', description=None, row_count=1185, column_count=7, stream_id=825, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={'calculation_ca9d4b1c-f73a-4f76-9f94-d3c4ca6871c5': {'templateId': 2664, 'id': 'calculation_ca9d4b1c-f73a-4f76-9f94-d3c4ca6871c5', 'name': 'rowcount', 'formula': 'sum(1)', 'status': 'VALID', 'dataType': 'LONG', 'persistedOnDataSource': True, 'isAggregatable': True, 'bignumber': False, 'variable': False}, 'calculation_38846559-d190-4ab1-809b-bcd361db5670': {'templateId': 2665, 'id': 'calculation_38846559-d190-4ab1-809b-bcd361db5670', 'name': 'max_views', 'formula': 'max(views)', 'status': 'VALID', 'dataType': 'LONG', 'persistedOnDataSource': True, 'isAggregatable': True, 'bignumber': False, 'columnPositions': [{'columnName': 'views', 'columnPosition': 4}], 'variable': False}}, schema=DomoDa

In [10]:
# | exporti
@patch_to(DomoDatacenter, cls_method=True)
async def get_accounts(
    cls,
    auth=dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    # can accept one value or a list of values
    additional_filters_ls=None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dma.DomoAccount]:
    """search Domo Datacenter account api.
    Note: at the time of this writing 7/18/2023, the datacenter api does not support searching accounts by name
    """

    json_list = await cls.search_datacenter(
        auth=auth,
        maximum=maximum,
        entity_type=datacenter_routes.Datacenter_Enum.ACCOUNT.value,
        additional_filters_ls=additional_filters_ls,
        return_raw=return_raw,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or len(json_list) == 0:
        return json_list

    domo_account_ls = [
        dma.DomoAccount._from_json(json_obj, auth=auth) for json_obj in json_list
    ]

    return domo_account_ls

#### sample implementation search_accounts


In [11]:
#| eval : false
# import domolibrary.routes.datacenter as datacenter_routes

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    datacenter_routes.generate_search_datacenter_filter(
        datacenter_routes.Datacenter_Filter_Field_Enum.DATAPROVIDER.value,
        "Google Sheets",
    )
]

(await DomoDatacenter.get_accounts(
    auth=auth,
    # additional_filters_ls=additional_filters_ls,
    return_raw=False,
    debug_api=False,
))[0:5]

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoAccount(id='50', name='**ReplicationUtility**:modocorp.domo.com', data_provider_type='dataset-copy', created_dt=datetime.datetime(2022, 10, 6, 13, 20, 34), modified_dt=datetime.datetime(2022, 10, 6, 13, 20, 34), config=None, owners=[DictDot(id='1345102715', type='USER', displayName='McWayne Mumford')], is_admin_summary=True),
 DomoAccount(id='84', name='**ReplicationUtility**:domo-ncv.domo.com', data_provider_type='dataset-copy', created_dt=datetime.datetime(2023, 11, 20, 15, 25, 42), modified_dt=datetime.datetime(2023, 11, 20, 15, 25, 42), config=None, owners=[DictDot(id='152768850', type='USER', displayName='Heath Pulham')], is_admin_summary=True),
 DomoAccount(id='45', name='onyxReporting@gmail.com', data_provider_type='google-spreadsheets', created_dt=datetime.datetime(2022, 10, 4, 16, 59, 14), modified_dt=datetime.datetime(2022, 10, 4, 16, 59, 14), config=None, owners=[DictDot(id='1893952720', type='USER', displayName='Jae Wilson')], is_admin_summary=True),
 DomoAccount(id='5

In [12]:
# | exporti


class LineageTypes_Enum(Enum):
    DomoDataset = "DATA_SOURCE"
    DomoDataflow = "DATAFLOW"


@patch_to(DomoDatacenter, cls_method=True)
async def get_lineage_upstream(
    cls,
    auth: dmda.DomoAuth,
    domo_entity,  # DomoDataset or DomoDataflow
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_prn: bool = False,
):

    import domolibrary.classes.DomoDataset as dmds
    import domolibrary.classes.DomoDataflow as dmdf

    if not session:
        session = httpx.AsyncClient()
        is_close_session = True

    res = await datacenter_routes.get_lineage_upstream(
        auth=auth,
        entity_type=LineageTypes_Enum[domo_entity.__class__.__name__].value,
        entity_id=domo_entity.id,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or res.status != 200:
        await session.aclose()
        return res

    obj = res.response

    domo_obj = []
    for key, item in obj.items():
        if item.get("type") == "DATA_SOURCE":
            domo_obj.append(
                await dmds.DomoDataset.get_from_id(
                    auth=auth, dataset_id=item.get("id"), session=session
                )
            )

        if item.get("type") == "DATAFLOW":
            # print(item.get('id'))
            domo_obj.append(
                await dmdf.DomoDataflow.get_from_id(
                    auth=auth, dataflow_id=item.get("id"), session=session
                )
            )
            pass
    
    if is_close_session:
        await session.aclose()
    return domo_obj

In [13]:
#| eval : false
# import domolibrary.classes.DomoDataset as dmds

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataset = dmds.DomoDataset(id="45a7a24e-c738-44f9-9019-00d5a33613ae")

await DomoDatacenter.get_lineage_upstream(
    auth=auth,
    domo_entity=domo_dataset,
    # return_raw = True,
    # debug_api = True
)

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoDataset(id='45a7a24e-c738-44f9-9019-00d5a33613ae', display_type='dataflow', data_provider_type='dataflow', name='sys_date', description=None, row_count=7299, column_count=9, stream_id=None, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=['s_domostats', 'domoStats', 'domostats', 'c_people']), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object at 0x7f2b4b12d330>),
 DomoDataflow(id=2, name='sys_numbers', owner=None, description=None, tags=None, actions=[DomoDataflow_Action(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='LoadFromVault', name='people', datasource_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None, depends_on=None, parent_actions=None), DomoDataflow_Action(id='51e63668-5ccd-46db-8180-554909545f38', type='GenerateTableAction', name='num', datasource_id=None, sql='SELECT\ncast( p0.n\n+ p1.n*2\n+ p2.n 

In [14]:
# | exporti
@patch_to(DomoDatacenter, cls_method=True)
async def search_cards(
    cls,
    auth=dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    search_text=None,
    # can accept one value or a list of values
    additional_filters_ls=None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dmds.DomoDataset]:

    json_list = await cls.search_datacenter(
        auth=auth,
        maximum=maximum,
        search_text=search_text,
        entity_type=datacenter_routes.Datacenter_Enum.CARD.value,
        additional_filters_ls=additional_filters_ls,
        return_raw=return_raw,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or len(json_list) == 0:
        return json_list

    return await ce.gather_with_concurrency(
        n=60,
        *[
            dmc.DomoCard.get_by_id(
                card_id=json_obj.get("databaseId"),
                auth=auth,
                debug_api=debug_api,
                session=session,
            )
            for json_obj in json_list
        ]
    )

In [15]:
#| eval : false
# from domolibrary.routes.datacenter import (
#     generate_search_datacenter_filter,
#     Datacenter_Filter_Field_Enum,
#     Datacenter_Filter_Field_Certification_Enum,
# )

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    generate_search_datacenter_filter(
        field=Datacenter_Filter_Field_Enum.CERTIFICATION.value,
        value=Datacenter_Filter_Field_Certification_Enum.CERTIFIED.value,
    )
]

print(additional_filters_ls)

await DomoDatacenter.search_cards(
    auth=auth,
    additional_filters_ls=additional_filters_ls,
    maximum=5,
    # return_raw = True,
    # debug_api = True
)

[{'filterType': 'term', 'field': 'certification.state', 'value': 'CERTIFIED', 'not': False}]
warning this token has not been validated by who_am_i, run get_auth_token first


[DomoCard(id=1036070144, title='Salesforce', description=None, type='kpi', urn='1036070144', chart_type='badge_pie', dataset_id='fff33fa8-bc96-4754-92ad-5a2f8789717f', certification=DictDot(state='CERTIFIED', lastUpdated=1705541792599, userId='1734948150', processType='DEPARTMENT', approvalRequestId='37bb5013-8939-4cc7-9e46-ad2299832040', processName='MajorDomo - TB', adminCertified=False), owners=[DomoUser(id='1734948150', display_name='Thomas Bruno', email_address='Thomas.Bruno@elevancehealth.com', role_id=2097317660, department=None, title=None, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=DomoRole(id=2097317660, name='Community_Default_Priviliged', description='Full access except for editing users and settings owners', is_system_role=0, is_default_role=None, grant_ls=[], m

In [16]:
# | exporti


@patch_to(DomoDatacenter, cls_method=True)
async def get_cards_admin_summary(
    cls,
    auth=dmda.DomoAuth,
    page_ids: List[str] = None,
    card_search_text: str = None,
    page_search_text: str = None,
    maximum: int = None,  # maximum number of results to return
    # can accept one value or a list of values
    return_raw: bool = False,
    debug_api: bool = False,
    debug_loop: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dmc.DomoCard]:
    """search Domo Datacenter card api."""

    search_body = card_routes.generate_body_search_cards_admin_summary(
        page_ids=page_ids,
        card_search_text=card_search_text,
        page_search_text=page_search_text,
    )

    res = await card_routes.search_cards_admin_summary(
        auth=auth,
        body=search_body,
        maximum=maximum,
        debug_api=debug_api,
        debug_loop=debug_loop,
        session=session,
        wait_sleep=5,
    )

    if return_raw or len(res.response) == 0:
        return res

    domo_account_ls = await ce.gather_with_concurrency(
        n=60,
        *[dmc.DomoCard._from_json(json_obj, auth=auth) for json_obj in res.response]
    )

    return domo_account_ls

In [17]:
#| eval : false
auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDatacenter.get_cards_admin_summary(
    auth=auth,
    # return_raw = True,
    maximum=5,
    # debug_api = True
)

warning this token has not been validated by who_am_i, run get_auth_token first


[DomoCard(id=332819959, title=' Forecast-Sales_MG', description=None, type='kpi', urn=None, chart_type=None, dataset_id=None, certification=None, owners=[DomoUser(id='722128535', display_name='Miguel Guedes', email_address='Miguel.guedes@cgi.com', role_id=2, department=None, title=None, avatar_key='ef7c051e-2b2f-4ad4-844b-bf348853beb6', phone_number='+351913017621', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None, custom_attributes={}, role=DomoRole(id=2, name='Privileged', description='Full access except for editing users and settings owners', is_system_role=True, is_default_role=None, grant_ls=[], membership_ls=[]))]),
 DomoCard(id=1582016299, title=" Incorrect Append - Forecast Column Header Names Don't Match", description=None, type='document', urn=None, chart_type=None, dataset_id=None, certification=None, owners=[DomoUser(id='583215149'

In [18]:
# | hide
import nbdev

nbdev.nbdev_export()